In [1]:
# Import libraries
import pandas as pd
from bs4 import BeautifulSoup as bs
from splinter import Browser
import requests
import time

In [2]:
# function to visit links
# Parameters url: url to visit
#            seconds: seconds to wait before scraping
#            browser: instance of the browser
# Returns: browser.html
def visit_url(url,seconds,browser):
    browser.visit(url)
    time.sleep(seconds)
    #print(browser.html)
    return browser.html

In [3]:
# Set up the chromedriver and browser
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
browser.driver.set_window_size(1920, 1080, browser.driver.window_handles[0])

year_list = ["2012","2013","2014","2015","2016","2017","2018","2019"]

df_dog_lyme_data = pd.DataFrame(columns=["year","county","positive_cases","total_tested"])

prev_positive_test = 0
prev_total_tested = 0

# loop from 2012 to 2019
for year in year_list:
    state_url = "https://capcvet.org/maps/#" + year + "/all/lyme-disease/dog/united-states/new-jersey"
    print(state_url)
    soup = bs(visit_url(state_url,3,browser), 'html.parser')

    browser.driver.execute_script("location.reload(true);")
    browser.execute_script("window.scrollTo(0, 250);")  
    browser.find_by_id('zoom_out').click()
    time.sleep(3)
    results = browser.find_by_xpath("//*[name()='path'][@class='county-land']")
    for result in results:
        result.click()
        time.sleep(3)
        soup = bs(browser.html,'html.parser')
        location = soup.find_all('ul')
        location1 = location[23].find_all('li')
        location2 = location1[2].find('span')
        numbers = soup.find_all('data')
        positive_test = numbers[2].text.replace(",","")
        total_tested = numbers[3].text.replace(",","")
        if prev_positive_test == positive_test and prev_total_tested == total_tested:
            positive_test = 0
            total_tested = 0

        row = [year,location2.text,positive_test,total_tested]
        df_dog_lyme_data.loc[len(df_dog_lyme_data)] = row

        prev_positive_test = positive_test
        prev_total_tested = total_tested
        
df_dog_lyme_data.to_csv("nj_capcvet.csv", index=False)
df_dog_lyme_data

browser.quit()


C:\Users\nickv\Anaconda3\envs\PythonData2020\lib\site-packages\selenium\webdriver\remote\webdriver.py:1089: UserWarning: Only 'current' window is supported for W3C compatibile browsers.
  warnings.warn("Only 'current' window is supported for W3C compatibile browsers.")


https://capcvet.org/maps/#2012/all/lyme-disease/dog/united-states/new-jersey
https://capcvet.org/maps/#2013/all/lyme-disease/dog/united-states/new-jersey
https://capcvet.org/maps/#2014/all/lyme-disease/dog/united-states/new-jersey
https://capcvet.org/maps/#2015/all/lyme-disease/dog/united-states/new-jersey
https://capcvet.org/maps/#2016/all/lyme-disease/dog/united-states/new-jersey
https://capcvet.org/maps/#2017/all/lyme-disease/dog/united-states/new-jersey
https://capcvet.org/maps/#2018/all/lyme-disease/dog/united-states/new-jersey
https://capcvet.org/maps/#2019/all/lyme-disease/dog/united-states/new-jersey


,year,county,positive_cases,total_tested
0,2012,Sussex County,372,1302
1,2012,Passaic County,102,637
2,2012,Bergen County,667,8900
3,2012,Warren County,1444,6225
4,2012,Morris County,1175,6567
...,...,...,...,...
163,2019,Gloucester County,601,10575
164,2019,Salem County,62,1556
165,2019,Atlantic County,582,7671
166,2019,Cumberland County,191,923
